In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import plotly.express as px
# from geopy.geocoders import Nominatim 
# from geopy.distance import geodesic

# Данные

In [43]:
path = '../data/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [44]:
df.drop(['Unnamed: 0','Тип', 'Телефоны', 'Ссылка на объявление', 'Серия дома' ], axis= 1, inplace=True)

In [45]:
# убираем адреса не в Мск
df = df[df['Адрес'].str.contains('Москва')]
# Оставляем только те, где цена указана в рублях
# df = df[df['Цена'].str.contains('руб.')]


In [46]:
# для подсчета количества пропущенных значений в каждом столбце
df.iloc[:, 7:12].isna().sum()

Цена                     0
Описание                 0
Ремонт                2463
Площадь комнат, м2    7228
Балкон                6630
dtype: int64

In [47]:
df.isna().sum()

ID  объявления                  0
Количество комнат             535
Метро                         346
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    11174
Цена                            0
Описание                        0
Ремонт                       2463
Площадь комнат, м2           7228
Балкон                       6630
Окна                         5150
Санузел                      2041
Можно с детьми/животными     4915
Дополнительно                 272
Название ЖК                 15281
Высота потолков, м           9202
Лифт                         4192
Мусоропровод                 8007
dtype: int64

# Релиз 2.0

### Метро

In [48]:
df['Метро'].isna()
df[['Метро', 'Адрес']]

,Метро,Адрес
0,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27"
1,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27"
2,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27"
3,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1"
4,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15"
...,...,...
23363,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ..."
23364,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс..."
23365,м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11"
23366,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ..."


### Площадь комнат

In [49]:
# вытаскиваем данные о площади комнат из площади квартиры по 2 индексу 
room_areas = df['Площадь, м2'].str.split('/').apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
df.loc[df['Площадь комнат, м2'].isna(), 'Площадь комнат, м2'] = room_areas
missing_indices = df['Площадь комнат, м2'].isna()
df.loc[missing_indices, 'Площадь комнат, м2'] = df.loc[missing_indices, 'Площадь, м2'].str.split('/').str[0]


In [50]:
df['Площадь комнат, м2'].isna().sum()

0

### Количество комнат

In [51]:
df['Количество комнат'].value_counts()


Количество комнат
1                   6646
2, Изолированная    4149
2                   2228
3                   1463
3, Изолированная    1404
3, Оба варианта      721
4                    601
2, Смежная           571
2, Оба варианта      459
5                    214
4, Оба варианта      209
4, Изолированная     188
6                     83
3, Смежная            80
5, Оба варианта       75
5, Изолированная      40
6, Оба варианта       26
6, Изолированная      16
4, Смежная            13
1, Изолированная       6
5, Смежная             4
6, Смежная             3
1, Оба варианта        2
1, Смежная             1
Name: count, dtype: int64

In [52]:
df['Количество комнат'].describe()

count     19202
unique       24
top           1
freq       6646
Name: Количество комнат, dtype: object

In [53]:
df['Количество комнат'] = df['Количество комнат'].apply(lambda x: int(x[0]) if type(x) is str else x)
df2= df['Площадь, м2'].apply(lambda x: float(x[:x.find('/')]) if type(x) is str else x)

def get_room_num(size): 
    if size < 42.5: return 1
    elif size < 64: return 2
    elif size < 108.5: return 3
    else: return 4
df['Количество комнат'] = df['Количество комнат'].fillna(df2.apply(get_room_num))
df['Количество комнат'].isna().value_counts()


Количество комнат
False    19737
Name: count, dtype: int64

In [54]:
df['Количество комнат'].isna().sum()

0

### Парковка

In [55]:
df['Парковка'].isna().sum()

11174

In [56]:
df['Парковка'].value_counts()

Парковка
наземная          5283
подземная         2466
открытая           713
многоуровневая     100
на крыше             1
Name: count, dtype: int64

In [57]:
# Если в столбце 'Название ЖК' есть записи, заполняем пропущенные значения в столбце 'Парковка' значением 'есть'
df.loc[df['Название ЖК'].notnull() & df['Парковка'].isna(), 'Парковка'] = 'есть'

# Заполняем пропущенные значения в столбце 'Парковка' значением 'нет'

df['Парковка'] = df['Парковка'].fillna('нет')
df['Парковка'].isna().sum()

0

In [58]:
df['Парковка'].value_counts()

Парковка
нет               9345
наземная          5283
подземная         2466
есть              1829
открытая           713
многоуровневая     100
на крыше             1
Name: count, dtype: int64

In [59]:
# Заменяем все значения, кроме 'нет', на 'есть'
# df['Парковка'] = df['Парковка'].replace(['наземная', 'подземная', 'есть', 'открытая', 'многоуровневая', 'на крыше'], 'есть')
# df['Парковка'].value_counts() 

### Ремонт

In [60]:
df['Ремонт'].value_counts()

Ремонт
Косметический    7361
Евроремонт       7004
Дизайнерский     2773
Без ремонта       136
Name: count, dtype: int64

In [61]:
# Заменим на моду 
df['Ремонт'] = df['Ремонт'].fillna(df['Ремонт'].mode()[0])
df.fillna({'Ремонт': df['Ремонт'].mode()[0]}, inplace=True)

In [62]:
# или заменим на Не указан
# df['Ремонт']= df['Ремонт']fillna('Не указано')

In [63]:
df['Ремонт'].isna().sum()

0

### Балкон 

In [64]:
df['Балкон'].describe()

count          13107
unique            18
top       Балкон (1)
freq            6336
Name: Балкон, dtype: object

In [65]:
#Количество пропусков в столбце с Балконом
df['Балкон'].isna().value_counts()

Балкон
False    13107
True      6630
Name: count, dtype: int64

In [66]:
df['Балкон'].value_counts()

Балкон
Балкон (1)                6336
Лоджия (1)                5032
Балкон (1), Лоджия (1)     610
Лоджия (2)                 537
Балкон (2)                 423
Балкон (3)                  45
Лоджия (3)                  42
Балкон (1), Лоджия (2)      21
Балкон (2), Лоджия (2)      21
Балкон (2), Лоджия (1)      16
Лоджия (4)                   5
Балкон (3), Лоджия (1)       4
Балкон (4)                   4
Балкон (1), Лоджия (3)       4
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [67]:
#Функция для заполнения пропусков в столбце Балкон на самые часто встречающиеся значение - Балкон(1) и Лоджия(1)
def notna_balcony(row):
    if pd.isna(row['Балкон']): 
        floor = int(re.findall(r'\d+', row['Дом'])[0])  # Получаем первый найденный этаж
        type_of_house = row['Дом'].split(',')[-1].strip()  # Убираем лишние пробелы
        area = float(row['Площадь, м2'].split('/')[0])
        if floor > 3 and area > 100 and type_of_house != 'Монолитный':
            return 'Балкон (1)'
        else:
            return 'Лоджия (1)'
    else:
        return row['Балкон']
    
df['Балкон'] = df.apply(notna_balcony, axis=1)
df['Балкон'].isna().sum()

0

### Окна

In [68]:
#Сколько раз встречаются уникальные значения в столбце Окна
df['Окна'].value_counts()

Окна
Во двор            9708
На улицу и двор    2762
На улицу           2117
Name: count, dtype: int64

In [69]:
#Количество пропусков в столбце Окна
df['Окна'].isna().value_counts()

Окна
False    14587
True      5150
Name: count, dtype: int64

In [70]:
#Согласно анализу объявлений, если в описании присутствует фраза "вид на", то, как правило вид из окон на достопримечательность, что сыграет роль в стоимости аренды
#Функция для заполнения значений по описанию:
def from_description(row):
    if pd.isna(row['Окна']) and 'вид на' in row['Описание'].lower():
        return 'На достопримечательность'
    
    elif pd.isna(row['Окна']) and 'вид во' in row['Описание'].lower():
        return 'Во двор'
    else:
        return row['Окна']

df['Окна'] = df.apply(from_description, axis=1)

In [71]:
df['Окна'].value_counts()

Окна
Во двор                     9712
На улицу и двор             2762
На улицу                    2117
На достопримечательность     138
Name: count, dtype: int64

In [72]:
#Функция для заполнения пропусков в столбце Окна в зависимости от этажа
def from_floor(row):
    if pd.isna(row['Окна']): 
        floor = int(re.findall(r'\d+', row['Дом'])[0])
        if floor < 6:
            return 'Во двор'
    else:
        return row['Окна']    
 
df['Окна'] = df.apply(from_floor, axis=1) 

In [73]:
#Окончательно заполняем пропуски:

# Вычисляем частоту встречания каждого значения в столбце "Окна"
window_counts = df['Окна'].value_counts(normalize=True)

# Заполняем пропуски в столбце "Окна" значениями на основе их частоты встречания
df['Окна'] = df['Окна'].fillna(pd.Series(np.random.choice(window_counts.index, p=window_counts, size=len(df))))
df['Окна'].value_counts()

Окна
Во двор                     13704
На улицу и двор              3138
На улицу                     2404
На достопримечательность      147
Name: count, dtype: int64

### Санузел

In [74]:
df['Санузел'].value_counts()

Санузел
Совмещенный (1)                    8500
Раздельный (1)                     6139
Совмещенный (2)                    1293
Совмещенный (1), Раздельный (1)     730
Раздельный (2)                      401
Совмещенный (3)                     223
Совмещенный (2), Раздельный (1)     179
Совмещенный (4)                      74
Раздельный (3)                       44
Совмещенный (1), Раздельный (2)      28
Совмещенный (3), Раздельный (1)      26
Совмещенный (2), Раздельный (2)      22
Раздельный (4)                       15
Совмещенный (3), Раздельный (3)       6
Совмещенный (4), Раздельный (1)       6
Совмещенный (4), Раздельный (2)       4
Совмещенный (1), Раздельный (3)       2
Совмещенный (2), Раздельный (3)       2
Совмещенный (2), Раздельный (4)       1
Совмещенный (3), Раздельный (2)       1
Name: count, dtype: int64

In [75]:
#Заполним пропуски модой при группировке по количеству комнат:
mode_per_room = df.groupby('Количество комнат')['Санузел'].transform(lambda x: x.mode().iloc[0])
df['Санузел'] = df['Санузел'].fillna(mode_per_room)

In [76]:
df.columns

Index(['ID  объявления', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'Парковка', 'Цена', 'Описание', 'Ремонт', 'Площадь комнат, м2',
       'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными',
       'Дополнительно', 'Название ЖК', 'Высота потолков, м', 'Лифт',
       'Мусоропровод'],
      dtype='object')

### Можно с детьми/животными

In [77]:
df['Можно с детьми/животными'].isna().value_counts()

Можно с детьми/животными
False    14822
True      4915
Name: count, dtype: int64

In [78]:
df['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       8540
Можно с детьми, Можно с животными    6075
Можно с животными                     207
Name: count, dtype: int64

In [79]:
# Применяем функцию для заполнения пропусков в столбце "Можно с детьми/животными" на основе описания
def fill_children_animals_description(row):
    if pd.isna(row['Можно с детьми/животными']):
        description = row['Описание'].lower()
        if any(word in description for word in ['детск', 'ребен', 'школ', 'семь']):
            if 'нельзя с' not in description:
                return 'Можно с детьми'
            else:
                return 'Нельзя никого'
        elif any(word in description for word in ['питом', 'животн', 'собак', 'кошк', 'пес', 'кот']):
            return 'Можно с животными'
        elif 'нельзя с' in description:
            return 'Нельзя никого'
        else:
            return np.nan
    else:
        return row['Можно с детьми/животными']

df['Можно с детьми/животными'] = df.apply(fill_children_animals_description, axis=1)

In [80]:
# Применяем функцию для заполнения пропусков в столбце "Можно с детьми/животными" на основе площади квартиры
def fill_children_animals_area(row):
    if pd.isna(row['Можно с детьми/животными']):
        if float(row['Площадь, м2'].split('/')[0]) < 50:
            return 'Нельзя никого'
        else:
            return 'Можно с детьми'   #Оставшиеся значения заполняются самым часто встречающимся
    else:
        return row['Можно с детьми/животными']   
            

df['Можно с детьми/животными'] = df.apply(fill_children_animals_area, axis=1)

### Дополнительно

In [81]:
df['Дополнительно'].value_counts()

Дополнительно
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Телевизор, Холодильник, Интернет                                       2553
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Холодильник, Интернет                                                  1085
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник, Интернет     977
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Кондиционер, Телевизор, Холодильник, Интернет                           906
Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Телевизор, Холодильник, Интернет, Телефон                               859
                                                                                                                                     ... 
Стиральная машина, Кондиционер                                                                                                          1
Мебель в комнатах, М

In [82]:
df['Дополнительно'].isna().value_counts()

Дополнительно
False    19465
True       272
Name: count, dtype: int64

In [83]:
df[df['Описание'].str.contains('без мебели', case=False, na=False)]

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод
8,273973191,3.0,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Лот 71833. Евгений Николаев.\n\nБонус агенту 1...,Евроремонт,45 25 25,Балкон (1),На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,3.0,Пасс (1),Нет
19,274748017,5.0,м. Кропоткинская (6 мин пешком),"Москва, Филипповский переулок, 9",201.0/162.0/16.0,3/3,нет,"190000.0 руб./ За месяц, Залог - 190000 руб., ...",ID 10903: Просторная 5-комнатная квартира в д...,Евроремонт,24-22-24-28-64,Лоджия (1),На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,3.2,Пасс (1),NaN
56,274617827,5.0,м. Кропоткинская (8 мин пешком),"Москва, Филипповский переулок, 9",200.0/150.0/30.0,"3/3, Кирпичный",нет,"190000.0 руб./ За месяц, Залог - 190000 руб., ...",Предлагается 5-ти комнатная квартира в отреста...,Евроремонт,30-30-30-30-30,Лоджия (1),Во двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Стиральная машина, Телевизор,...",NaN,NaN,NaN,NaN
69,271264562,5.0,м. Кропоткинская (5 мин пешком),"Москва, Гоголевский бульвар, 23",170.0/104.0/21.0,"5/5, старый фонд",наземная,"175000.0 руб./ За месяц, Залог - 175000 руб., ...",Лот 67428. Анастасия Баталова.\n\nБонус агенту...,Евроремонт,14 11 20 48 11,Балкон (1),На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,3.2,Пасс (1),Да
74,271766412,6.0,м. Александровский сад (4 мин пешком),"Москва, переулок Романов, 5",228.0/159.0/30.0,3/6,нет,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Лот 30273. Коллегам бонус 50 000 рублей. Татья...,Евроремонт,50+30+19+18+23+19,Балкон (1),Во двор,Совмещенный (2),Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,Пасс (1),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,274204547,4.0,м. Сокол (5 мин на машине),"Москва, Иваньковское шоссе, 5",150.0/83.0/15.0,"16/23, Монолитно-кирпичный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 81033. Ирина Волкова.\n\nБонус агенту 2500...,Евроремонт,19 18 11 35,Балкон (1),На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,3.0,"Пасс (1), Груз (1)",Да
23031,274776175,4.0,м. Щукинская (19 мин пешком),"Москва, Иваньковское шоссе, 5",150.0/100.0/20.0,16/24,нет,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 69725. Просторная светлая квартира без меб...,Евроремонт,25+20+20+35,Балкон (1),На улицу,Совмещенный (2),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Стиральная...",NaN,NaN,NaN,NaN
23132,269188642,4.0,м. Стрешнево (10 мин на машине),"Москва, Никольский тупик, 2к1",220.0/130.0/20.0,"4/6, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ЭЛИТНЫЙ ЖК ЧАЙКА с круглосуточной охраной, вид...",Дизайнерский,130.0,Балкон (1),Во двор,Совмещенный (3),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,NaN,"Пасс (2), Груз (2)",NaN
23133,271992616,4.0,м. Сокол (10 мин на машине),"Москва, Береговая улица, 8К1",203.0/92.0/25.0,1/5,наземная,"600000.0 руб./ За месяц, Залог - 600000 руб., ...","ID 35735: Предлагается превосходная, просторн...",Евроремонт,30-20-20-22,Лоджия (1),На улицу,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель на кухне, Ванна, Душевая кабина, Стирал...",NaN,3.2,Пасс (1),NaN


In [84]:
# Применяем функцию для заполнения пропусков в столбце на основе описания
def additional_description(row):
    if pd.isna(row['Дополнительно']):
        description = row['Описание'].lower()
        if any(word in description for word in ['без мебели', 'мебели нет']):
            if any(word in description for word in ['стиральная', 'техникой', 'холодильник', 'оборудована']):
                return 'мебель и техника'  # Обновляем значение на "мебель и техника"
            else:
                return 'ничего'  # Обновляем значение на "ничего"
        elif any(word in description for word in ['частично мебел', 'вся мебель', 'мебель в', 'необходимая', 'необходимой', 'меблирован', 'новой мебелью']):
            if any(word in description for word in ['стиральная', 'техникой', 'холодильник', 'оборудована']):
                return 'мебель и техника'  # Обновляем значение на "мебель и техника"
            else:
                return 'мебель'  # Обновляем значение на "мебель"
        else:
            return 'мебель и техника'
    else:
        return row['Дополнительно']

df['Дополнительно'] = df.apply(additional_description, axis=1)

In [85]:
# #Приведение названий столбцов к нужномуу формату - английский язык + соединение слов _РЕШИЛИ ДЕЛАТЬ В КОНЦЕ,,,
df.rename(columns = {'Площадь комнат, м2': 'rooms_square', 'Балкон': 'balcony', 'Окна': 'windows', 'Санузел': 'bathroom', 'Можно с детьми/животными': 'сhildren/pets_allowed',
                    'Дополнительно': 'additionally'}, inplace=True)

### Проверяем наличие пропусков 

In [86]:
df.isna().sum()

ID  объявления               0
Количество комнат            0
Метро                      346
Адрес                        0
Площадь, м2                  0
Дом                          0
Парковка                     0
Цена                         0
Описание                     0
Ремонт                       0
rooms_square                 0
balcony                      0
windows                    344
bathroom                     0
сhildren/pets_allowed        0
additionally                 0
Название ЖК              15281
Высота потолков, м        9202
Лифт                      4192
Мусоропровод              8007
dtype: int64

# Релиз 3.0

### Цена 

In [87]:
# преобразование цены 
# в руб
def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [88]:
# срок аренды
def extract_rent_term(text):
    match = re.search(r'Срок аренды - (Длительный|На несколько месяцев)', text)
    if match:
        return match.group(1)
    else:
        return None

In [89]:
# добавляем столбцы Цена за мес, залог и срок аренды
df['Срок аренды'] = df['Цена'].apply(extract_rent_term)
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)
df.head(3) 

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Цена за месяц,Залог,Ремонт,rooms_square,balcony,windows,bathroom,сhildren/pets_allowed,additionally,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,Срок аренды
0,271271157,4.0,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,500000,500000,Дизайнерский,20.0,Лоджия (1),Во двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",3.0,"Пасс (4), Груз (1)",Да,Длительный
1,271634126,4.0,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,500000,500000,Дизайнерский,25 25 20 25,Балкон (1),На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.5,"Пасс (1), Груз (1)",Нет,Длительный
2,271173086,4.0,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",500000,500000,Евроремонт,116.0,Балкон (1),На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.2,Пасс (1),NaN,Длительный


In [90]:
df['Срок аренды'].value_counts()

Срок аренды
Длительный              19185
На несколько месяцев      552
Name: count, dtype: int64

### Парковка

In [91]:
df['Парковка'].value_counts()

Парковка
нет               9345
наземная          5283
подземная         2466
есть              1829
открытая           713
многоуровневая     100
на крыше             1
Name: count, dtype: int64

### Количество комнат

### Этаж

In [92]:
# преобразование этажа и этажности дома
def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])

### Общая площадь

In [93]:
# Общая площадь
def get_area_lamb(s):
    if '/' in s:
        # Если в строке есть '/', используем для извлечения первой части перед '/'
        return float(s.split('/')[0])
    else:
        # Если нет '/', преобразуем значение в число
        return float(s)

def get_area(data):
    return df['Площадь, м2'].astype(str).apply(get_area_lamb)

# Создаем новый столбец 
df['Общая площадь'] = get_area(df)

# Станция

In [94]:
# вытаскиваем Станцию
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].astype(str).map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [95]:
# конвертируем расстояние до метро на пешком

def metro_time_lamb(s):
    try:
        if 'пешком' in s:
            # Если в строке содержится 'пешком'
            return float(s.split('(')[1].split(' мин')[0].strip()) if 'None' not in s else 0
        else:
            # Если не указано 'пешком', предполагаем, что это время на транспорте (умножаем на 10 для преобразования)
            return float(s.split('(')[1].split(' мин')[0].strip()) * 10 if 'None' not in s else 0

    except:
        pass

def get_metro_time(data):
    return df['Метро'].apply(metro_time_lamb)

# Создаем новый столбец 'metro_time' и заполняем его значениями, полученными из функции get_metro_time
df['Время пешком до метро, мин'] = get_metro_time(df)

#### Метро 


In [96]:
metro = pd.read_csv('../data/list_of_moscow_metro_stations.csv', sep=',').rename(columns={'Station':'Станция'})
metro.head()

,Станция,metro,coor
0,Третьяковская,"Москва, станция метро Третьяковская","(55.738862, 37.6284034)"
1,Медведково,"Москва, станция метро Медведково","(55.886766, 37.6615351)"
2,Первомайская,"Москва, станция метро Первомайская","(55.7931157, 37.7999286)"
3,Калужская,"Москва, станция метро Калужская","(55.6550976, 37.5421849)"
4,Каховская,"Москва, станция метро Каховская","(55.6536252, 37.6054067)"


In [97]:
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19737 entries, 0 to 23367
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID  объявления              19737 non-null  int64  
 1   Количество комнат           19737 non-null  float64
 2   Метро                       19391 non-null  object 
 3   Станция                     19737 non-null  object 
 4   Адрес                       19737 non-null  object 
 5   Площадь, м2                 19737 non-null  object 
 6   Дом                         19737 non-null  object 
 7   Парковка                    19737 non-null  object 
 8   Цена                        19737 non-null  object 
 9   Описание                    19737 non-null  object 
 10  Цена за месяц               19737 non-null  int64  
 11  Залог                       19737 non-null  int64  
 12  Ремонт                      19737 non-null  object 
 13  rooms_square                19737 no

In [98]:
#координаты

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])

In [99]:
df

,ID объявления,Количество комнат,Метро,metro_coord,Станция,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Цена за месяц,Залог,Ремонт,rooms_square,balcony,windows,bathroom,сhildren/pets_allowed,additionally,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,Срок аренды,Этаж,Этажность дома,Общая площадь,"Время пешком до метро, мин"
0,271271157,4.0,м. Смоленская (9 мин пешком),"(55.7488802, 37.5830392)",смоленская,"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,500000,500000,Дизайнерский,20.0,Лоджия (1),Во двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",3.00,"Пасс (4), Груз (1)",Да,Длительный,5,16,200.0,9.0
1,271634126,4.0,м. Смоленская (8 мин пешком),"(55.7488802, 37.5830392)",смоленская,"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,500000,500000,Дизайнерский,25 25 20 25,Балкон (1),На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.50,"Пасс (1), Груз (1)",Нет,Длительный,5,16,198.0,8.0
2,271173086,4.0,м. Смоленская (7 мин пешком),"(55.7488802, 37.5830392)",смоленская,"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",500000,500000,Евроремонт,116.0,Балкон (1),На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.20,Пасс (1),NaN,Длительный,5,16,200.0,7.0
3,272197456,4.0,м. Смоленская (3 мин пешком),"(55.7488802, 37.5830392)",смоленская,"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,400000,400000,Евроремонт,14-42-20-19,Балкон (1),На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,3.20,Пасс (1),NaN,Длительный,5,6,170.0,3.0
4,273614615,2.0,м. Арбатская (7 мин пешком),"(55.7522109, 37.6087395)",арбатская,"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",есть,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,225000,225000,Евроремонт,20 18,Лоджия (1),На улицу и двор,Совмещенный (2),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,3.90,"Пасс (1), Груз (1)",Да,Длительный,12,26,58.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1.0,м. Говорово (8 мин пешком),NaN,говорово,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,есть,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",Сдаётся светлая теплая квартира (студия) с бал...,42000,42000,Евроремонт,16.4,Балкон (1),NaN,Совмещенный (1),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,"Пасс (1), Груз (1)",NaN,Длительный,10,14,35.0,8.0
23364,274654844,1.0,м. Солнцево (7 мин пешком),NaN,солнцево,"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",есть,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",Сдам однокомнатную квартиру на длительный срок...,45000,45000,Евроремонт,16.5,Лоджия (1),Во двор,Совмещенный (1),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,"Пасс (1), Груз (1)",NaN,Длительный,5,18,38.7,7.0
23365,268679909,2.0,м. Солнцево (6 мин пешком),NaN,солнцево,"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",нет,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",Рассматриваются квартиранты только славяне и т...,50000,50000,Дизайнерский,43.1,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухн